# [Santa25] Simulated annealing with translations

In this notebook, we are going to perform simulated anealing using two trees which are translated in $x$ and $y$ directions. Length of translation in $x$ and $y$ directions are generally not the same as well as the number of translations. See notebook for more details.

Basic idea for this approach was shared by **hengck23** [here](https://www.kaggle.com/competitions/santa-2025/discussion/637061#3345209)

Codes (or parts of them) from the following notebooks are used here:
- [Santa 2025 - Getting Started](https://www.kaggle.com/code/inversion/santa-2025-getting-started)
- [[86.08]exp-annealing-hybrid](https://www.kaggle.com/code/argon1215/86-08-exp-annealing-hybrid)
- [Santa 2025 - simple, but long running optimization](https://www.kaggle.com/code/chistyakov/santa-2025-simple-but-long-running-optimization)

In [ ]:
import datetime
import copy
import math
import random
import time
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

In [ ]:
getcontext().prec = 25
scale_factor = Decimal("1e15")

In [ ]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x="0", center_y="0", angle="0"):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (
                    -(trunk_w / Decimal("2")) * scale_factor,
                    trunk_bottom_y * scale_factor,
                ),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

    def get_params(self):
        return self.center_x, self.center_y, self.angle

    def set_params(self, center_x, center_y, angle):
        self.__init__(str(center_x), str(center_y), str(angle))

    def clone(self) -> "ChristmasTree":
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

In [ ]:
def format_time(elapsed):
    """Take a time in seconds and return a string hh:mm:ss."""
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
def plot_trees(n, trees, score):
    for t in trees:
        plt.plot(*t.polygon.exterior.xy)
    plt.axis("equal")
    plt.title(f"n={n}\nScore: {score:.4f}")
    plt.show()

In [ ]:
def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

In [ ]:
def has_collision(trees: list[ChristmasTree]) -> bool:
    """Check for collisions between trees"""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

In [ ]:
class SimulatedAnnealing:
    def __init__(
        self,
        trees,
        nt,
        Tmax,
        Tmin,
        nsteps,
        nsteps_per_T,
        cooling,
        alpha,
        n,
        position_delta,
        angle_delta,
        delta1,
        random_state,
        log_freq,
    ):
        self.trees = trees
        self.nt = nt
        self.Tmax = Tmax
        self.Tmin = Tmin
        self.nsteps = nsteps
        self.nsteps_per_T = nsteps_per_T
        self.cooling = cooling
        self.alpha = alpha
        self.n = n
        self.position_delta = position_delta
        self.angle_delta = angle_delta
        self.delta1 = delta1
        self.log_freq = log_freq
        random.seed(random_state)

    def perturb_tree(self, tree):
        """Perturb tree position and angle"""
        old_x, old_y, old_angle = tree.get_params()
        dx = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dy = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dangle = Decimal(str(random.uniform(-self.angle_delta, self.angle_delta)))
        new_x = old_x + dx
        new_y = old_y + dy
        new_angle = (old_angle + dangle) % 360
        tree.set_params(new_x, new_y, new_angle)
        return old_x, old_y, old_angle

    def get_length(self, current_trees):
        xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in current_trees])
        min_x, min_y = xys.min(axis=0)
        max_x, max_y = xys.max(axis=0)
        length = max(max_x - min_x, max_y - min_y)

        lengthx = length.copy()
        lengthy = length.copy()
        while True:
            trees_ = self.translate(current_trees, lengthx - self.delta1, length, [2, 1])
            if has_collision(trees_):
                break
            else:
                lengthx -= self.delta1
            while True:
                trees_ = self.translate(current_trees, length, lengthy - self.delta1, [1, 2])
                if has_collision(trees_):
                    break
                else:
                    lengthy -= self.delta1
        return lengthx, lengthy

    def translate(self, current_trees, lengthx, lengthy, nt):
        trees_ = []
        for tree in current_trees:
            for x in range(nt[0]):
                for y in range(nt[1]):
                    trees_.append(
                        ChristmasTree(
                            center_x=tree.center_x + Decimal(x * lengthx),
                            center_y=tree.center_y + Decimal(y * lengthy),
                            angle=tree.angle,
                        )
                    )
        return trees_

    def solve(self):

        t0 = time.time()  # Measure staring time

        T = self.Tmax

        current_trees = copy.deepcopy(self.trees)

        lengthx, lengthy = self.get_length(current_trees)
        trees_ = self.translate(current_trees, lengthx, lengthy, self.nt)

        current_score = calculate_score(trees_)
        best_trees = copy.deepcopy(current_trees)
        best_score = current_score

        for step in range(self.nsteps):
            for step1 in range(self.nsteps_per_T):
                i = random.randint(0, len(current_trees) - 1)
                old_params = self.perturb_tree(current_trees[i])

                lengthx, lengthy = self.get_length(current_trees)
                trees_ = self.translate(current_trees, lengthx, lengthy, self.nt)

                if has_collision(trees_):
                    current_trees[i].set_params(*old_params)
                    if step1 % self.log_freq == 0 or step1 == (self.nsteps_per_T - 1):
                        t1 = format_time(time.time() - t0)
                        print(
                            f"T: {T:.3e}  Step: {step1:6}  Score: {current_score:8.5f}  Best score: {best_score:8.5f}  Elapsed Time: {t1}",
                            flush=True,
                        )
                    continue

                new_score = calculate_score(trees_)
                delta = new_score - current_score

                if delta < 0 or random.random() < math.exp(-delta / T):
                    current_score = new_score
                    if new_score < best_score:
                        best_score = new_score
                        best_trees = copy.deepcopy(trees_)
                        print(f"NEW BEST SCORE: {best_score:8.5f}")
                else:
                    current_trees[i].set_params(*old_params)

                if step1 % self.log_freq == 0 or step1 == (self.nsteps_per_T - 1):
                    t1 = format_time(time.time() - t0)
                    print(
                        f"T: {T:.3e}  Step: {step1:6}  Score: {current_score:8.5f}  Best score: {best_score:8.5f}  Elapsed Time: {t1}",
                        flush=True,
                    )

            # lower the temperature
            if self.cooling == "linear":
                T -= (self.Tmax - self.Tmin) / self.nsteps
            elif self.cooling == "exponential":
                Tfactor = -math.log(self.Tmax / self.Tmin)
                T = self.Tmax * math.exp(Tfactor * (step + 1) / self.nsteps)
            elif self.cooling == "polynomial":
                T = self.Tmin + (self.Tmax - self.Tmin) * ((self.nsteps - step - 1) / self.nsteps) ** self.n

        return best_score, best_trees

In [ ]:
initial_trees = []
for x, y, deg in [-2.93069232,-4.24856960,67], [-3.92971914, -4.16631769, 250.00]:
    initial_trees.append(ChristmasTree(x, y, deg))

for t in initial_trees:
    plt.plot(*t.polygon.exterior.xy)
plt.axis("equal")
plt.title("Initial trees")
plt.show()

In [ ]:
%%writefile config.yaml

n: 2
csv: submission.csv
max_attempts: 100

params:
    nt: [4, 9]
    Tmax: 0.0002
    Tmin: 0.00005
    alpha: 0.99
    nsteps: 15
    nsteps_per_T: 500
    cooling: 'exponential'
    alpha: 0.99
    n: 4
    position_delta: 0.01
    angle_delta: 30.0
    delta1: 0.01
    random_state: 42
    log_freq: 250

In [ ]:
with open("config.yaml", "r") as file_obj:
    config = yaml.safe_load(file_obj)

sa = SimulatedAnnealing(initial_trees, **config["params"])
score, trees_72 = sa.solve()

plot_trees(72, trees_72, score)

In [ ]:
config["params"]["nt"] = [5, 10]

sa = SimulatedAnnealing(initial_trees, **config["params"])
score, trees_100 = sa.solve()

plot_trees(100, trees_100, score)

In [ ]:
config["params"]["nt"] = [5, 11]

sa = SimulatedAnnealing(initial_trees, **config["params"])
score, trees_110 = sa.solve()

plot_trees(110, trees_110, score)

In [ ]:
config["params"]["nt"] = [6, 12]

sa = SimulatedAnnealing(initial_trees, **config["params"])
score, trees_144 = sa.solve()

plot_trees(144, trees_144, score)

In [ ]:
config["params"]["nt"] = [6, 13]

sa = SimulatedAnnealing(initial_trees, **config["params"])
score, trees_156 = sa.solve()

plot_trees(156, trees_156, score)

In [ ]:
config["params"]["nt"] = [7, 14]

sa = SimulatedAnnealing(initial_trees, **config["params"])
score, trees_196 = sa.solve()

plot_trees(196, trees_196, score)

In [ ]:
config["params"]["nt"] = [7, 15]

sa = SimulatedAnnealing(initial_trees, **config["params"])
score, trees_210 = sa.solve()

plot_trees(210, trees_210, score)

trees_200 = trees_210[:200].copy()
plot_trees(200, trees_200, 0.)

In [ ]:
new_trees = {72: trees_72, 100: trees_100, 110: trees_110, 144: trees_144, 156: trees_156, 196: trees_196, 200: trees_200}

In [ ]:
def load_configuration_from_df(n, existing_df):
    """
    Load existing configuration from submission CSV.
    """
    group_data = existing_df[existing_df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = row["x"][1:]  # Remove 's' prefix
        y = row["y"][1:]
        deg = row["deg"][1:]
        trees.append(ChristmasTree(x, y, deg))
    if len(trees) != n:
        raise RuntimeError("Number of trees is inconsistent")
    return trees

In [ ]:
def to_str(x: Decimal):
    return f"s{float(x)}"

In [ ]:
df = pd.read_csv("/kaggle/input/why-not/submission.csv")

rows = []
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if n in new_trees:
        for i_t, tree in enumerate(new_trees[n]):
            rows.append(
                {
                    "id": f"{n:03d}_{i_t}",
                    "x": to_str(tree.center_x),
                    "y": to_str(tree.center_y),
                    "deg": to_str(tree.angle),
                }
            )
    else:
        for i_t, tree in enumerate(trees):
            rows.append(
                {
                    "id": f"{n:03d}_{i_t}",
                    "x": to_str(tree.center_x),
                    "y": to_str(tree.center_y),
                    "deg": to_str(tree.angle),
                }
            )

df = pd.DataFrame(rows)
df.to_csv("submission.csv", index=False)

In [ ]:
def get_tree_list_side_lenght(tree_list: list[ChristmasTree]) -> Decimal:
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_total_score(dict_of_side_length: dict[str, Decimal]):
    score = 0
    for k, v in dict_of_side_length.items():
        score += v ** 2 / Decimal(k)
    return score

def parse_csv(csv_path) -> dict[str, list[ChristmasTree]]:

    result = pd.read_csv(csv_path)
    result['x'] = result['x'].str.strip('s')
    result['y'] = result['y'].str.strip('s')
    result['deg'] = result['deg'].str.strip('s')
    result[['group_id', 'item_id']] = result['id'].str.split('_', n=2, expand=True)

    dict_of_tree_list = {}
    dict_of_side_length = {}
    for group_id, group_data in result.groupby('group_id'):
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg']) for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
        dict_of_side_length[group_id] = get_tree_list_side_lenght(tree_list)

    return dict_of_tree_list, dict_of_side_length


# Load current best solution
current_solution_path = 'submission.csv'
dict_of_tree_list, dict_of_side_length = parse_csv(current_solution_path)

# Calculate current total score
current_score = get_total_score(dict_of_side_length)


for group_id_main in range(200, 1, -1):
    group_id_main = f'{int(group_id_main):03n}'
    print(f'Current box: {group_id_main}')

    group_id_prev = f'{int(group_id_main) - 1:03n}'
    best_side_length = dict_of_side_length[group_id_prev]
    best_tree_to_delete = None
    
    # Try to delete each tree one by one and select the best option
    for tree_to_delete in range(int(group_id_main)):
        candidate_tree_list = [tree.clone() for tree in dict_of_tree_list[group_id_main]]
        del candidate_tree_list[tree_to_delete]

        candidate_side_length = get_tree_list_side_lenght(candidate_tree_list)

        if candidate_side_length < best_side_length:
            print(f' improvement {best_side_length:0.8f} -> {candidate_side_length:0.8f}')
            best_side_length = candidate_side_length
            best_tree_to_delete = tree_to_delete

    # Save the best
    if best_tree_to_delete is not None:
        candidate_tree_list = [tree.clone() for tree in dict_of_tree_list[group_id_main]]
        del candidate_tree_list[best_tree_to_delete]
        
        dict_of_tree_list[group_id_prev] = candidate_tree_list
        dict_of_side_length[group_id_prev] = get_tree_list_side_lenght(candidate_tree_list)
    
# Recalculate current total score
new_score = get_total_score(dict_of_side_length)
print(f'{current_score=:0.8f} {new_score=:0.8f} ({current_score - new_score:0.8f})')

# Save results
tree_data = []
for group_name, tree_list in dict_of_tree_list.items():
    for item_id, tree in enumerate(tree_list):
        tree_data.append({
            'id': f'{group_name}_{item_id}',
            'x': f's{tree.center_x}',
            'y': f's{tree.center_y}',
            'deg': f's{tree.angle}'
        })
tree_data = pd.DataFrame(tree_data)
tree_data.to_csv('submission.csv', index=False)